In [ ]:
import torch.nn as nn
import torch
import torch.nn.functional as F

class GoogLeNet(nn.Module):
    def __init__(self, num_classes=40, aux_logits=True, init_weights=True):
        super(GoogLeNet,self).__init__()
        self.aux_logits = aux_logits

        self.conv1 = BasicConv1d(1, 64, kernel_size=7, stride=2, padding=3)    
        self.maxpool1 = nn.MaxPool1d(3, stride=2, ceil_mode=True)              

        self.conv2 = BasicConv1d(64, 64, kernel_size=1)
        self.conv3 = BasicConv1d(64, 192, kernel_size=3, padding=1)
        self.maxpool2 = nn.MaxPool1d(3, stride=2, ceil_mode=True)

        self.inception3a = Inception(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = Inception(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = nn.MaxPool1d(3, stride=2, ceil_mode=True)

        self.inception4a = Inception(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = Inception(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = Inception(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = Inception(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = Inception(528, 256, 160, 320, 32, 128, 128)
        self.maxpool4 = nn.MaxPool1d(3, stride=2, ceil_mode=True)

        self.inception5a = Inception(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = Inception(832, 384, 192, 384, 48, 128, 128)

        if self.aux_logits:
            self.aux1 = InceptionAux(512, num_classes)
            self.aux2 = InceptionAux(528, num_classes)

        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(1024, num_classes)
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.maxpool2(x)

        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool3(x)
        x = self.inception4a(x)
        if self.training and self.aux_logits: 
            aux1 = self.aux1(x)

        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        if self.training and self.aux_logits:  
            aux2 = self.aux2(x)

        x = self.inception4e(x)
        x = self.maxpool4(x)
        x = self.inception5a(x)
        x = self.inception5b(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)
        if self.training and self.aux_logits: 
            return x, aux2, aux1
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

class Inception(nn.Module):
    def __init__(self, in_channels, ch1x1, ch3x3red, ch3x3, ch5x5red, ch5x5, pool_proj):
        super(Inception,self).__init__()

        self.branch1 = BasicConv1d(in_channels, ch1x1, kernel_size=1)

        self.branch2 = nn.Sequential(
            BasicConv1d(in_channels, ch3x3red, kernel_size=1),
            BasicConv1d(ch3x3red, ch3x3, kernel_size=3, padding=1)         
        )

        self.branch3 = nn.Sequential(
            BasicConv1d(in_channels, ch5x5red, kernel_size=1),
            BasicConv1d(ch5x5red, ch5x5, kernel_size=5, padding=2)        
        )

        self.branch4 = nn.Sequential(
            nn.MaxPool1d(kernel_size=3, stride=1, padding=1),              
            BasicConv1d(in_channels, pool_proj, kernel_size=1)
        )

    def forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)

        outputs = [branch1, branch2, branch3, branch4]
        return torch.cat(outputs, 1)

class InceptionAux(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(InceptionAux,self).__init__()
        self.averagePool = nn.AvgPool1d(kernel_size=5, stride=3)
        self.conv = BasicConv1d(in_channels, 128, kernel_size=1)        

        self.fc1 = nn.Linear(3840, 1024)
        self.fc2 = nn.Linear(1024,num_classes)

    def forward(self, x):
        x = self.averagePool(x)
        x = self.conv(x)
        x = torch.flatten(x, 1)

        x = F.dropout(x, 0.5, training=self.training)
        x = F.relu(self.fc1(x), inplace=True)
        x = F.dropout(x, 0.5, training=self.training)
        x = self.fc2(x)
        return x

class BasicConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(BasicConv1d, self).__init__()
        self.conv = nn.Conv1d(in_channels, out_channels, **kwargs)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        return x

In [7]:
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np

def plot_matrix(conf_matrix, dev_list, save_path):
    plt.figure(figsize=(20, 16), dpi=300)
    plt.imshow(conf_matrix, cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()

    thresh = conf_matrix.max() / 2.
    for i in range(conf_matrix.shape[0]):
        for j in range(conf_matrix.shape[1]):
            plt.text(j, i, conf_matrix[i, j],
                     ha="center", va="center",
                     color="white" if conf_matrix[i, j] > thresh else "black", fontsize=6)

    tick_marks = np.arange(len(dev_list))
    plt.xticks(tick_marks, dev_list)
    plt.yticks(tick_marks, dev_list)
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.savefig(save_path)
    plt.show()

In [8]:
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.categories = sorted(os.listdir(data_dir))
        self.data = []
        self.transform = transform
        for category in self.categories:
            category_dir = os.path.join(data_dir, category)
            category_data = sorted(os.listdir(category_dir))
            self.data.extend([(os.path.join(category_dir, file), self.categories.index(category)) for file in category_data])

    def __getitem__(self, index):
        file_path, label = self.data[index]
        data = np.load(file_path)
        image = Image.fromarray(data.astype(np.uint8))
        image = transform(image)
        return image, label

    def __len__(self):
        return len(self.data)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("using {} device.".format(device))

transform = transforms.Compose([ transforms.Grayscale(num_output_channels = 1),
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.5), (0.5))])

train_dataset = CustomDataset("features/train_npy",transform=transform)
train_num = len(train_dataset)
dev_list = train_dataset.categories

batch_size = 32
train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size, shuffle=True,
                                               num_workers=0)

validate_dataset = CustomDataset("features/val_npy",transform=transform)
val_num = len(validate_dataset)
validate_loader = torch.utils.data.DataLoader(validate_dataset,
                                                  batch_size=batch_size, shuffle=False,
                                                  num_workers=0)

print("using {} images for training, {} images for validation.".format(train_num, val_num))

net = GoogLeNet(num_classes=31, aux_logits=True, init_weights=True)
net.to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)

epochs = 100
save_path = './GoogLeNet_parameters.pth'
best_f1 = 0.0
train_accurate_list = []
val_accurate_list = []
f1_list = []
recall_list = []

for epoch in range(epochs):
    net.train()
    running_loss = 0.0
    train_acc = 0.0
    for step, data in enumerate(train_loader, start=0):
        images, labels = data
        images = images.reshape(images.shape[0], 1, 1500)
        optimizer.zero_grad()
        logits, aux_logits2, aux_logits1 = net(images.to(device))
        predict_y = torch.max(logits, dim=1)[1]
        train_acc += torch.eq(predict_y, labels.to(device)).sum().item()
        loss0 = loss_function(logits, labels.to(device))
        loss1 = loss_function(aux_logits1, labels.to(device))
        loss2 = loss_function(aux_logits2, labels.to(device))
        loss = loss0 + loss1 * 0.3 + loss2 * 0.3
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        rate = (step + 1) / len(train_loader)
        a = "*" * int(rate * 50)
        b = "." * int((1 - rate) * 50)
        print("\rtrain loss:{:^3.0f}%[{}->{}]{:.3f}".format(int(rate * 100), a, b, loss), end="")
    print()
    train_accurate = train_acc / train_num
    train_accurate_list.append(train_accurate)
    net.eval()
    acc = 0.0  
    val = torch.tensor([])
    pre = torch.tensor([])
    with torch.no_grad():
        for val_data in validate_loader:
            val_images, val_labels = val_data
            val_images = val_images.reshape(val_images.shape[0], 1, 1500)
            outputs = net(val_images.to(device))
            predict_y = torch.max(outputs, dim=1)[1]
            pre = torch.cat([pre.to(device), predict_y.to(device)])
            val = torch.cat([val.to(device), val_labels.to(device)])
            acc += torch.eq(predict_y, val_labels.to(device)).sum().item()
    val_accurate = acc / val_num
    val_accurate_list.append(val_accurate)
    f1 = f1_score(val.cpu(), pre.cpu(), average='macro')
    recall = recall_score(val.cpu(), pre.cpu(), average='macro')

    f1_list.append(f1)
    recall_list.append(recall)
    if f1 > best_f1:
        best_f1 = f1
        best_pre = pre
        best_val = val
        torch.save(net.state_dict(), save_path)
        torch.save(best_pre, 'pre_val_label/best_pre_GoogLeNet.pt')
        torch.save(best_val, 'pre_val_label/best_val_GoogLeNet.pt')
    print('[epoch %d] train_loss: %.3f train_accuracy: %.3f val_accuracy: %.3f  recall: %.3f  f1: %.3f' %
              (epoch + 1, running_loss / step, train_accurate, val_accurate, recall, f1))
    with open("GoogLeNet_result_npy.txt", 'a') as file:
        file.write("[epoch " + str(epoch + 1) + "]" + "  " + "train_accuracy:" + str(train_accurate) + "  " + "val_accuracy:" + str(val_accurate) + "  " + "recall:" + str(recall) + "  " + "f1:" + str(f1) + '\n')
print('Finished Training')
iterations = range(1, len(train_accurate_list) + 1)
with open("GoogLeNet_npy_plt_data.txt", 'a') as file:
    file.write("iterations:" + str(iterations) +
               "train_accurate_list:" + str(train_accurate_list) +
               "val_accurate_list:" + str(val_accurate_list) +
               "f1_list:" + str(f1_list) +
               "recall_list:" + str(recall_list) +
               "dev_list:" + str(dev_list) + '\n')
conf_matrix = confusion_matrix(best_val.cpu(),best_pre.cpu())
plot_matrix(conf_matrix,dev_list,"GoogLeNet_confusion_matrix_npy.png")

using cuda:0 device.
using 86137 images for training, 21515 images for validation.
train loss:100%[**************************************************->]2.482
[epoch 1] train_loss: 3.608 train_accuracy: 0.307 val_accuracy: 0.431  recall: 0.144  f1: 0.128
train loss:100%[**************************************************->]1.663
[epoch 2] train_loss: 2.459 train_accuracy: 0.490 val_accuracy: 0.525  recall: 0.246  f1: 0.231
train loss:100%[**************************************************->]1.647
[epoch 3] train_loss: 1.992 train_accuracy: 0.562 val_accuracy: 0.597  recall: 0.368  f1: 0.367
train loss:100%[**************************************************->]1.354
[epoch 4] train_loss: 1.726 train_accuracy: 0.612 val_accuracy: 0.643  recall: 0.479  f1: 0.446
train loss:100%[**************************************************->]1.527
[epoch 5] train_loss: 1.512 train_accuracy: 0.658 val_accuracy: 0.687  recall: 0.542  f1: 0.533
train loss:100%[**********************************************

train loss:100%[**************************************************->]0.117
[epoch 96] train_loss: 0.127 train_accuracy: 0.978 val_accuracy: 0.949  recall: 0.901  f1: 0.898
train loss:100%[**************************************************->]0.049
[epoch 97] train_loss: 0.125 train_accuracy: 0.979 val_accuracy: 0.955  recall: 0.891  f1: 0.887
train loss:100%[**************************************************->]0.096
[epoch 98] train_loss: 0.124 train_accuracy: 0.979 val_accuracy: 0.950  recall: 0.887  f1: 0.895
train loss:100%[**************************************************->]0.235
[epoch 99] train_loss: 0.130 train_accuracy: 0.977 val_accuracy: 0.955  recall: 0.904  f1: 0.901
train loss:100%[**************************************************->]0.205
[epoch 100] train_loss: 0.119 train_accuracy: 0.979 val_accuracy: 0.951  recall: 0.894  f1: 0.897
Finished Training
